In [1]:
import pandas as pd
import pickle
import os

from utils_mitgcm import open_mitgcm_ds_from_config

In [2]:
model = 'geneva_200m'
params_file = 'swirl_03'
pickle_file_name = f'{model}_{params_file}_day1.pkl'
output_folder = r'../Outputs'

output_filename = f'{model}_{params_file}_day1_lvl0.csv'

# Open MITgcm results

In [3]:
mitgcm_config, ds_mitgcm = open_mitgcm_ds_from_config('../config.json', model)

# Open pickle eddies file

In [4]:
input_file_path = os.path.join(output_folder, pickle_file_name)
with open(input_file_path, 'rb') as f:
    eddies = pickle.load(f)

# Create catalogue lvl0

In [8]:
def compute_ke_snapshot(snapshot_vel):
    ke = 0.5 * (snapshot_vel.UVEL.values**2 + snapshot_vel.VVEL.values**2 + snapshot_vel.WVEL.values**2) * snapshot_vel.dxC.values * snapshot_vel.dyC.values * snapshot_vel.drC.values  # This gives J per cell
    
    return ke / 1e6  # Convert to MJ

In [9]:
def translate_rotation_direction(eddy_orientation: int):
    return 'clockwise' if eddy_orientation == -1 else 'anticlockwise' 

In [10]:
# Helper function to extract eddy info into a row
def extract_eddy_data(id_level0, indices_eddy, eddy, snapshot_vel, ke_snapshot_megajoules, surface_cell):
    vortex_indices = tuple(eddy.vortex_cells.astype(int))
    ke_eddy = ke_snapshot_megajoules[vortex_indices[1], vortex_indices[0]].sum()
    surface_area = len(eddy.vortex_cells[0]) * surface_cell

    return {
        'id': id_level0,
        'time_index': indices_eddy[0],
        'depth_index': indices_eddy[1],
        'eddy_index': indices_eddy[2],
        'date': pd.Timestamp(snapshot_vel.time.values).to_pydatetime(),
        'depth_[m]': float(snapshot_vel.Z.values),
        'xc': eddy.center[0],
        'yc': eddy.center[1],
        'surface_area_[m2]': float(surface_area),
        'volume_slice_[m3]': float(surface_area * float(snapshot_vel.drC.values)),
        'rotation_direction': translate_rotation_direction(eddy.orientation),
        'kinetic_energy_[MJ]': float(ke_eddy),
        'i_eddy_cells': eddy.vortex_cells[0],
        'j_eddy_cells': eddy.vortex_cells[1]
    }

In [11]:
# Initialize list to collect all DataFrame rows
eddy_rows = []
id_level0 = 0
for t_index in eddies['time_indices']:
    for d_index in eddies['depth_indices']:
        snapshot_vel = ds_mitgcm.isel(time=t_index, Z=d_index, Zp1=d_index, Zl=d_index, Zu=d_index)
        snapshot_eddies = eddies[t_index][d_index]
        surface_cell = snapshot_vel.dxC.values[0][0] * snapshot_vel.dyC.values[0][0]
        ke_snapshot_megajoules = compute_ke_snapshot(snapshot_vel)

        for eddy_index in range(len(snapshot_eddies)):
            indices_eddy = (t_index,d_index,eddy_index)
            row_data = extract_eddy_data(id_level0, indices_eddy, snapshot_eddies[eddy_index], snapshot_vel, ke_snapshot_megajoules, surface_cell)
            eddy_rows.append(row_data)
            id_level0 += 1

# Create final DataFrame in one step using pd.concat
df_catalogue_level0 = pd.concat([pd.DataFrame([row]) for row in eddy_rows], ignore_index=True)

# Save as CSV

In [12]:
df_catalogue_level0.to_csv(os.path.join(output_folder, output_filename), index=False)